In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# INSTALL & IMPORT LIBRARY

In [ ]:
!pip install underthesea
!pip install transformers
!pip install vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 40.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for v

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import re
# from underthesea import word_tokenize
from keras.utils import to_categorical
from transformers import AutoTokenizer
from tensorflow.data import Dataset
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
import pandas as pd

In [ ]:
from vncorenlp import VnCoreNLP


# PREPROCESSING

## LOAD DATA

In [ ]:
class TextNormalize:
    def __init__(self):
        self.vowels_to_ids = {}
        self.vowels_table = [
            ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
            ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
            ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
            ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
            ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
            ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
            ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
            ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'o'],
            ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
            ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
            ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
            ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
        ]
        pass

    def createVowelsTable(self):
        """Create Vowels Table"""
        for i in range(len(self.vowels_table)):
            for j in range(len(self.vowels_table[i]) - 1):
                self.vowels_to_ids[self.vowels_table[i][j]] = (i, j)

    def IsValidVietnameseWord(self,word):
        """Nguyên âm chỉ có thể đứng chung với nguyên âm. Một từ không thể có 2 nguyên âm cách nhau bởi 1 phụ âm"""
        chars = list(word)
        #nguyen am
        vowel_index = -1
        for i in range(len(chars)):
            idx_vowel_table = self.vowels_to_ids.get(chars[i],(-1,-1))[0]
            if idx_vowel_table != -1:
                if vowel_index == -1:
                    vowel_index = i
                else:
                    if i - vowel_index != 1:
                        return False
                    vowel_index = i
        return True

    def WordStandardized(self,word):
        """Standardize Word"""
        if not self.IsValidVietnameseWord(word):
            return word

        chars = list(word)
        vowel_indexes = []

        # tìm vị trí nguyên âm
        qu_or_gi = False
        thanh_dieu = 0
        for i in range(len(chars)):
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[i],(-1,-1))
            if vowel_table_row == -1 :
                continue
            # qu
            if vowel_table_row == 9:
                if i != 0 and chars[i-1] == 'q':
                    chars[i] = 'u'
                    qu_or_gi = True
            # gi
            elif vowel_table_row == 5:
                if i != 0 and chars[i-1] == 'g':
                    chars[i] = 'i'
                    qu_or_gi = True

            # có chứa thanh điệu
            if vowel_table_col != 0:
                thanh_dieu = vowel_table_col
                chars[i] = self.vowels_table[vowel_table_row][0]

            vowel_indexes.append(i)
        # 1 nguyên âm
        if len(vowel_indexes) == 1:
            c = chars[vowel_indexes[0]]
            chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
            return ''.join(chars)

        for idx_vowel in vowel_indexes:
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[idx_vowel],(-1,-1))
            #ê, ơ, ô
            if vowel_table_row == 4 or vowel_table_row == 7 or vowel_table_row == 8:
                c = chars[idx_vowel]
                chars[idx_vowel] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # kiểm tra qu và gi, 2-3 nguyên âm thì nguyên âm thứ 2 chứa dấu
            if qu_or_gi:
                if len(vowel_indexes) == 2 or len(vowel_indexes) == 3:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # 2 nguyên âm
            if len(vowel_indexes) == 2:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[0]]
                    chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            elif len(vowel_indexes) == 3:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

        return ''.join(chars)

    def normalize(self,text):

        #Chuyen sang viet thuong
        text = text.lower()

        # Rút gọn từ kéo dài
        text = re.sub(r'(\w)\1+',r'\1',text)

        # xóa các emoji dư thừa
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'',text) # no emoji

        text = text.split()
        # chuẩn hóa thanh điệu
        for i in range(len(text)):
            text[i] = self.WordStandardized(text[i])

        text = ' '.join(text)

        # xóa space d
        text = re.sub(r"( )\1+",r'\1',text)
        text = re.sub(r"[:)^@!`~%;?(\+\-\'\"]+",r'',text)

        # remove hastag
        text = re.sub("(@[A-Za-z0-9]+)|(#[0-9A-Za-z]+)"," ", text)
        return text

In [ ]:
def convert_unicode(text):
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  char1252 = char1252.split('|')
  charutf8 = charutf8.split('|')

  dic = {}
  for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
  return re.sub(
      r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
      lambda x: dic[x.group()], text
)


In [ ]:
class LoadData():
    def __init__(self, file_path):
        self.file_path = file_path

        self.data = pd.read_json(file_path, lines = True)
        # self.data = self.data.dropna()
        self.X = []
        self.y = []
    def transform(self,x,label):
        y = []
        if len(label) == 0:
          return x,np.array(["O" for i in range(len(x.split()))])

        first_index = label[:,0].astype(int)
        second_index = label[:,1].astype(int)
        asp_cate_pola = label[:,2]
        s = 0
        a = ""
        # chay tu s -> first, cap nhat s
        for i in range(len(label)):

            front = x[s:first_index[i]]
            # print(first_index[i],second_index[i])
            middle = x[first_index[i]:second_index[i]]
            # print(x)
            # print(middle)
            s = second_index[i]

            a += front + " " + middle + " "
            y.extend(["O" for i in range(len(front.split()))])
            y.extend([f"B-{asp_cate_pola[i]}" if j == 0 else f"I-{asp_cate_pola[i]}" for j in range(len(middle.split()))])

        if s != len(x):
            a+= x[s:]
            y.extend(["O" for i in range(len(x[s:].split()))])

        # print(a)
        # for k, v in zip(a.split(),y):
        #     print(k,"=>",v)
        return a,np.array(y)

    def ExtractAspectTermPosition(self,span_labels):
        labels = []
        # print(aspectTerms)
        for ls in span_labels:
            start = ls[0]
            end = ls[1]
            asp_cate_pola = ls[2]

            labels.append([int(start),int(end), asp_cate_pola])
        return np.array(sorted(labels,key = lambda x: x[0]))

    def load(self,):
        _len = len(self.data)
        for i in range(_len):
            x = self.data.iloc[i,0].strip() #text
            span_labels = self.data.iloc[i,1] #label
            span_labels = np.array(sorted(span_labels,key = lambda x: x[0]))

            x,y = self.transform(x,span_labels)
            self.X.append(convert_unicode(x))
            self.y.append(y)

        return self.X,self.y

In [ ]:
data = LoadData("/content/drive/MyDrive/dataset/train.jsonl")
X_raw,y_raw = data.load()


In [ ]:
X_raw[:5],y_raw[:5]

([' Pin Sài tầm 50h cho pin 100/100 .  Camera ổn  ...  tất cả đều OK  ...  nhân viên thế giới di động trần văn thời cà mau nhiệt tình và vui vẻ ...chúc các ae sức khỏe tốt và phục ok hoài nha ....',
  ' Lag  và  hao pin  là cái tóm tắt về máy.  Sam làm tệ quá, không bằng mấy con tàu cùng phân khúc ',
  'Tất cả đều ổn ngoại trừ lúc  máy nóng lên thì pin tụt nhanh hơn tụt quần  haizz.  Cam chụp cũng gọi là tầm trung .  Nếu dùng lướt web bình thường thì có thể dùng được cả ngày ',
  'Ok mua máy ở TGDD  chính sách đổi trả rất tốt,rất yên tâm khi mua ở TGDD,quản lý ở 15A đường bà hôm phường 13 quận 6 nhẹ nhàng và vui tính,nhiên viên ai cũng nhiệt tình, hài lòng về cách chăm sóc khách hàng của các bạn !.',
  ' kiểu dáng thì đẹp,cầm chắc tay ,nhưng  loa nhỏ quá , nhân viên phục vụ rất nhiệt tình '],
 [array(['B-BATTERY#POSITIVE', 'I-BATTERY#POSITIVE', 'I-BATTERY#POSITIVE',
         'I-BATTERY#POSITIVE', 'I-BATTERY#POSITIVE', 'I-BATTERY#POSITIVE',
         'I-BATTERY#POSITIVE', 'O', 'B-CAMERA#

In [ ]:
z = pd.read_json("/content/drive/MyDrive/dataset/train.jsonl",lines = True)
print(z.iloc[10,:].labels)

[]


In [ ]:
for k,v in zip(X_raw[10].split(),y_raw[10]):
  print(k,"=>",v)

Chỉ => O
cần => O
pin => O
trâu => O
sóng => O
khỏe => O
Bắc => O
wifi => O
mạnh => O
là => O
OK => O
Chỉ => O
cần => O
vậy => O
pin => O
sử => O
dụng => O
2e2 => O
3 => O
ngày => O
là => O
đc => O


# ASPECT TERM EXTRACTION

In [ ]:
PRETRAINED_MODEL = "vinai/phobert-base-v2"
SEP = "</s>"
MAX_LEN = 256
BATCH_SIZE = 8

## ALIGN LABEL

In [ ]:
class AlignLabel():
  def __init__(self):
    pass
  def Convert2LabelPosition(self,label):
    labels_position = []
    lst_first_pos = np.array([i if "B-" in v else 0 for i,v in enumerate(label)])
    lst_first_pos = np.argwhere(lst_first_pos != 0).reshape(1,-1)[0]

    for i in range(len(lst_first_pos)):
      # if i reach last pos: label range should be (i,len(label))
      last_pos = lst_first_pos[i]
      if i == len(lst_first_pos) - 1:
        for j in range(lst_first_pos[i],len(label)):
          if "I-" in label[j]:
            last_pos = j
      else:
        for j in range(lst_first_pos[i],lst_first_pos[i+1]):
          if "I-" in label[j]:
            last_pos = j
      labels_position.append([lst_first_pos[i],last_pos,label[lst_first_pos[i]]])

    return np.array(labels_position)

  def segment_and_alignLabel(self,x,y,tokenizer, SEP):
    def segment_and_addSEP(seg,ismid):
      seg = tokenizer.tokenize(seg)
      seg = [" ".join(s) for s in seg]
      seg = " ".join(seg)
      return seg


    y_new = []
    label = self.Convert2LabelPosition(y)
    preprocess = TextNormalize()

    if len(label) == 0:
      x_temp = preprocess.normalize(x)
      x_temp = segment_and_addSEP(x_temp,False)
      x_temp = " ".join(x_temp.split())
      y_new = ["O" for i in range(len(x_temp.split()))]
      return  [x_temp,np.array(y_new)]

    first_index = label[:,0].astype(int)
    second_index = label[:,1].astype(int)
    asp_cate_pola = label[:,2]
    s = 0
    a = ""
    x = x.split()
    for i in range(len(label)):
        front = " ".join(x[s:first_index[i]])
        if first_index[i] == second_index[i]:
          middle = x[first_index[i]]
          s = second_index[i] + 1
        else:
          middle = " ".join(x[first_index[i]:second_index[i]+1])
          s = second_index[i]+1

        front = preprocess.normalize(front)
        middle = preprocess.normalize(middle)

        front = segment_and_addSEP(front,False)
        middle = segment_and_addSEP(middle,True )

        a += front + " " + middle + " "
        y_new.extend(["O" for i in range(len(front.split()))])

        if first_index[i] == second_index[i]:
          y_new.extend([f"B-{asp_cate_pola[i][2:]}"])
        else:
          y_new.extend([f"B-{asp_cate_pola[i][2:]}" if j == 0 else f"I-{asp_cate_pola[i][2:]}" for j in range(len(middle.split(" ")))])

    if s != len(x):
        enc = " ".join(x[s:])
        enc = preprocess.normalize(enc)
        enc = segment_and_addSEP(enc,False)
        a+= enc
        y_new.extend(["O" for i in range(len(enc.split()))])

    a = " ".join(a.split())
    return [a,np.array(y_new)]

  def tokenize_and_alignlabel(self,x,y,tag2idx,tokenizer):
    x = x.strip().split(" ")
    y_position = self.Convert2LabelPosition(y)
    if len(y_position) == 0:
      return np.zeros(MAX_LEN)

    first_index = y_position[:,0].astype(int)
    second_index = y_position[:,1].astype(int)
    asp_cate_pola = y_position[:,2]
    y_new = np.zeros(MAX_LEN)
    x_tokenize = []
    s = 0
    pre_len = 0
    y_position = 1
    for i in range(len(first_index)):
      front_len = len(tokenizer(" ".join(x[s:first_index[i]]),add_special_tokens = False)['input_ids'])
      y_new[y_position:y_position + front_len] = tag2idx["O"]
      y_position += front_len

      if first_index[i] == second_index[i]:
        words = tokenizer(x[first_index[i]],add_special_tokens = False)['input_ids']
        s = second_index[i] + 1
        # print(x[first_index[i]])

      else:
        # print(x[first_index[i]:second_index[i]+1],)
        words = tokenizer(" ".join(x[first_index[i]:second_index[i]+1]),add_special_tokens = False)['input_ids']
        s = second_index[i] + 1
      y_new[y_position] = tag2idx[f'B-{asp_cate_pola[i][2:]}']
      # print(words)
      if len(words) >= 2:
        y_new[y_position+1:y_position+1+len(words)-1] = tag2idx[f'I-{asp_cate_pola[i][2:]}'] # skip B-name position, subtract -1 because len(words) contain B-name

      y_position += len(words)

    tokenize_last_sents = tokenizer(" ".join(x[s:]),add_special_tokens = False)['input_ids']
    y_new[y_position:y_position + len(tokenize_last_sents) + 1] = tag2idx["O"]

    end_sep_position = y_position  + len(tokenizer(" ".join(x[s:])))

    y_new[0] = tag2idx['O']
    y_new[end_sep_position ] = tag2idx['O']

    return y_new

  def transform(self,x_raw,y_raw,rdrsegmenter,SEP):
    X = []
    Y = []
    for i in range(len(x_raw)):
      x, y = self.segment_and_alignLabel(x_raw[i],y_raw[i],rdrsegmenter,SEP)
      X.append(x)
      Y.append(y)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X,Y

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')


In [ ]:
align_label = AlignLabel()
X, y = align_label.transform(X_raw,y_raw,rdrsegmenter,SEP)

<ipython-input-12-ed114fe53922>:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


In [ ]:
te = "Mua được 1 tuần thấy máy quá OK Pin trâu , máy mượt , cảm biến vân tay nhạy , k nóng máy . Nhu cầu chơi game nên k cần gì hơn , liên quân vô nhanh gần ngang ip . Vs mức giá này mua con máy sài thấy quá hời luôn"
rdrsegmenter.tokenize(te)

[['Mua',
  'được',
  '1',
  'tuần',
  'thấy',
  'máy',
  'quá',
  'OK',
  'Pin',
  'trâu',
  ',',
  'máy',
  'mượt',
  ',',
  'cảm_biến',
  'vân',
  'tay',
  'nhạy',
  ',',
  'k',
  'nóng',
  'máy',
  '.'],
 ['Nhu_cầu',
  'chơi',
  'game',
  'nên',
  'k',
  'cần',
  'gì',
  'hơn',
  ',',
  'liên_quân',
  'vô',
  'nhanh',
  'gần',
  'ngang',
  'ip',
  '.'],
 ['Vs',
  'mức',
  'giá',
  'này',
  'mua',
  'con',
  'máy',
  'sài',
  'thấy',
  'quá',
  'hời',
  'luôn']]

In [ ]:
m = 7
print(X[m])
for k,v in zip(X[m].split(),y[m]):
  print(k,"=>",v)

mua được 1 tuần thấy máy quá ok pin trâu , máy mượt , cảm_biến vân tay nhạy , k nóng máy . nhu_cầu chơi game nên k cần gì hơn , liên_quân vô nhanh gần ngang ip . vs mức giá này mua con máy sài thấy quá hời luôn
mua => O
được => O
1 => O
tuần => O
thấy => O
máy => B-GENERAL#POSITIVE
quá => I-GENERAL#POSITIVE
ok => I-GENERAL#POSITIVE
pin => B-BATTERY#POSITIVE
trâu => I-BATTERY#POSITIVE
, => O
máy => B-PERFORMANCE#POSITIVE
mượt => I-PERFORMANCE#POSITIVE
, => O
cảm_biến => B-FEATURES#POSITIVE
vân => I-FEATURES#POSITIVE
tay => I-FEATURES#POSITIVE
nhạy => I-FEATURES#POSITIVE
, => O
k => B-PERFORMANCE#POSITIVE
nóng => I-PERFORMANCE#POSITIVE
máy => I-PERFORMANCE#POSITIVE
. => I-PERFORMANCE#POSITIVE
nhu_cầu => I-PERFORMANCE#POSITIVE
chơi => I-PERFORMANCE#POSITIVE
game => I-PERFORMANCE#POSITIVE
nên => I-PERFORMANCE#POSITIVE
k => I-PERFORMANCE#POSITIVE
cần => I-PERFORMANCE#POSITIVE
gì => I-PERFORMANCE#POSITIVE
hơn => I-PERFORMANCE#POSITIVE
, => I-PERFORMANCE#POSITIVE
liên_quân => I-PERFORMANCE#PO

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def getTag2idx():
  """
    This will return tag2idx, idx2tag
  """
  aspect = np.array(["SCREEN","CAMERA","FEATURES","BATTERY","PERFORMANCE","STORAGE","DESIGN","PRICE","GENERAL","SER&ACC"])
  func_add_pola = lambda aspect,pola: [aspect[i] + "#" + pola for i in range(len(aspect))]
  func_add_prefix = lambda aspect,prefix: [prefix + "-" + aspect[i] for i in range(len(aspect))]

  aspect_pos = func_add_pola(aspect,"POSITIVE")
  aspect_neu = func_add_pola(aspect,"NEUTRAL")
  aspect_neg = func_add_pola(aspect,"NEGATIVE")

  B_aspect_pos = func_add_prefix(aspect_pos,"B")
  B_aspect_neu = func_add_prefix(aspect_neu,"B")
  B_aspect_neg = func_add_prefix(aspect_neg,"B")

  I_aspect_pos = func_add_prefix(aspect_pos,"I")
  I_aspect_neu = func_add_prefix(aspect_neu,"I")
  I_aspect_neg = func_add_prefix(aspect_neg,"I")
  all_labels = np.concatenate([B_aspect_pos,B_aspect_neu,B_aspect_neg,I_aspect_pos,I_aspect_neu,I_aspect_neg])
  tag2idx = {v:i+1 for i,v in enumerate(all_labels)}
  tag2idx["O"] = 0
  idx2tag = {v:k for k,v in tag2idx.items()}
  return tag2idx, idx2tag

In [ ]:
tag2idx, idx2tag = getTag2idx()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
tokenizer.model_max_length

1000000000000000019884624838656

In [ ]:
X_train[281]

'mua máy từ tháng 1 tổng kết máy dùng ngon , lướt web nhanh , bắt wifi tốt , chụp ảnh, quay vieo ok , pin trâu chơi game từ sáng đến chiều tối mới phải sạc'

In [ ]:
q = 6
print(X_train[q],y_train[q])
te = align_label.tokenize_and_alignlabel(X_train[q],y_train[q],tag2idx,tokenizer)
te

ok máy đẹp , hiệu năng ăn đứt mấy con 835 android . có điều sạc pin k nhanh, bù lại pin cũng lâu . 2019 mua 7p sài tới 2021 đổi lên xs max là hợp lý ['O' 'B-DESIGN#POSITIVE' 'I-DESIGN#POSITIVE' 'O' 'B-PERFORMANCE#POSITIVE'
 'I-PERFORMANCE#POSITIVE' 'I-PERFORMANCE#POSITIVE'
 'I-PERFORMANCE#POSITIVE' 'I-PERFORMANCE#POSITIVE'
 'I-PERFORMANCE#POSITIVE' 'I-PERFORMANCE#POSITIVE'
 'I-PERFORMANCE#POSITIVE' 'O' 'B-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL'
 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL'
 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL'
 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL' 'I-BATTERY#NEUTRAL' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']


array([ 0.,  0.,  7., 37.,  0.,  0.,  5., 35., 35., 35., 35., 35., 35.,
       35., 35., 35., 35.,  0.,  0., 14., 44., 44., 44., 44., 44., 44.,
       44., 44., 44., 44., 44., 44.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [ ]:
X_train[q],y_train[q]

('ok máy đẹp , hiệu năng ăn đứt mấy con 835 android . có điều sạc pin k nhanh, bù lại pin cũng lâu . 2019 mua 7p sài tới 2021 đổi lên xs max là hợp lý',
 array(['O', 'B-DESIGN#POSITIVE', 'I-DESIGN#POSITIVE', 'O',
        'B-PERFORMANCE#POSITIVE', 'I-PERFORMANCE#POSITIVE',
        'I-PERFORMANCE#POSITIVE', 'I-PERFORMANCE#POSITIVE',
        'I-PERFORMANCE#POSITIVE', 'I-PERFORMANCE#POSITIVE',
        'I-PERFORMANCE#POSITIVE', 'I-PERFORMANCE#POSITIVE', 'O',
        'B-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL',
        'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL',
        'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL',
        'I-BATTERY#NEUTRAL', 'I-BATTERY#NEUTRAL', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], dtype='<U22'))

In [ ]:
for k,v in zip(tokenizer.convert_ids_to_tokens(tokenizer(X_train[q])['input_ids']),te):
  print(k,"=>",idx2tag[v])

<s> => O
▁ok => O
▁máy => B-DESIGN#POSITIVE
▁đẹp => I-DESIGN#POSITIVE
▁ => O
, => O
▁hiệu => B-PERFORMANCE#POSITIVE
▁năng => I-PERFORMANCE#POSITIVE
▁ăn => I-PERFORMANCE#POSITIVE
▁đ => I-PERFORMANCE#POSITIVE
ứ => I-PERFORMANCE#POSITIVE
t => I-PERFORMANCE#POSITIVE
▁mấy => I-PERFORMANCE#POSITIVE
▁con => I-PERFORMANCE#POSITIVE
▁ => I-PERFORMANCE#POSITIVE
835 => I-PERFORMANCE#POSITIVE
▁android => I-PERFORMANCE#POSITIVE
▁ => O
. => O
▁có => B-BATTERY#NEUTRAL
▁điều => I-BATTERY#NEUTRAL
▁s => I-BATTERY#NEUTRAL
ạc => I-BATTERY#NEUTRAL
▁pin => I-BATTERY#NEUTRAL
▁k => I-BATTERY#NEUTRAL
▁nhanh => I-BATTERY#NEUTRAL
, => I-BATTERY#NEUTRAL
▁bù => I-BATTERY#NEUTRAL
▁lại => I-BATTERY#NEUTRAL
▁pin => I-BATTERY#NEUTRAL
▁cũng => I-BATTERY#NEUTRAL
▁lâu => I-BATTERY#NEUTRAL
▁ => O
. => O
▁2019 => O
▁mua => O
▁7 => O
p => O
▁s => O
à => O
i => O
▁tới => O
▁2021 => O
▁đổi => O
▁lên => O
▁x => O
s => O
▁max => O
▁là => O
▁hợp => O
▁lý => O
</s> => O


## CONVERT TO PRETRAIN FORMAT

In [ ]:
def toPretrainFormat(x, y, tokenizer, batch_size,is_shuffle = False):
  tag2idx, idx2tag = getTag2idx()
  features = {k:[] for k in tokenizer.model_input_names}

  for i in range(len(x)):

    token = tokenizer(x[i],add_special_tokens = True,padding = 'max_length',max_length = MAX_LEN,truncation=True)
    for name in tokenizer.model_input_names:
      temp = np.asarray(token[name]).astype('float')
      features[name].append(temp)

  for k,v in features.items():
    features[k] = np.asarray(features[k])

  y_new = np.zeros((len(y),MAX_LEN)) # [CLS] token
  for i in range(len(y)):
      y_new[i] = AlignLabel().tokenize_and_alignlabel(x[i],y[i],tag2idx,tokenizer)

  y_new = [to_categorical(y_new[i],num_classes = len(tag2idx)) for i in range(len(y_new))]

  dataset = Dataset.from_tensor_slices((features, y_new))
  if is_shuffle:
    dataset = dataset.shuffle(buffer_size = len(y_new))
  dataset = dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

  return dataset


In [ ]:
trainning = toPretrainFormat(X_train,y_train,tokenizer,BATCH_SIZE,is_shuffle = True)
trainning_dev = toPretrainFormat(X_dev,y_dev,tokenizer,BATCH_SIZE,)


In [ ]:
for i in trainning.take(1):
  print(tokenizer.convert_ids_to_tokens(i[0]['input_ids'][3]))
  print(i)

['<s>', '▁đúng', '▁như', '▁bạn', '▁mùa', '▁tru', '▁nói', '▁khi', '▁xem', '▁phim', '▁hay', '▁đọc', '▁báo', '▁vu', 'ốt', '▁màn', '▁hình', '▁nó', '▁cứ', '▁giật', '▁giật', '▁nh', 'ức', '▁cả', '▁mắt', '▁biết', '▁vậy', '▁mua', '▁vivo', '▁y', '91', 'c', '▁có', '▁2,3', '90', 'k', '▁mà', '▁nhiều', '▁người', '▁dùng', '▁nói', '▁m', 'ượt', '▁mà', '▁lắm', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

## MODEL

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 26.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode


class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import transformers
from tensorflow.keras.layers import Input, Dropout, Dense,concatenate,Bidirectional,LSTM,TimeDistributed,Lambda
from tensorflow.keras.models import Model


In [ ]:
inputs = {
    "input_ids" : Input(shape = (MAX_LEN),dtype = 'int32', name = 'input_ids'),
    # "token_type_ids": Input(shape = (MAX_LEN),dtype = 'int32', name = 'token_type_ids'),
    "attention_mask": Input(shape = (MAX_LEN),dtype = 'int32', name = 'attention_mask')
}

model = transformers.TFAutoModelForTokenClassification.from_pretrained(PRETRAINED_MODEL,output_hidden_states=True,from_pt=True)
features = model(inputs)[-1]

concat = tf.concat(features[-2:],2)
dropout = Dropout(0.1) (concat)
bi_lstm = Bidirectional(LSTM(768*2,return_sequences=True),merge_mode='concat') (dropout)

output = TimeDistributed(Dense(len(tag2idx),activation = 'relu')) (bi_lstm)

crf = CRF(len(tag2idx))
output = crf(output)
model = Model(inputs = inputs,outputs = output)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForTokenClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model = pretrained_model(PRETRAINED_MODEL)
# model.summary()

All PyTorch model weights were used when initializing TFXLMRobertaForTokenClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 tfxlm_roberta_for_token_classi  TFTokenClassifierOu  277454594  ['attention_mask[0][0]',         
 fication_2 (TFXLMRobertaForTok  tput(loss=None, log              'input_ids[0][0]']              
 enClassification)              its=(None, 256, 2),                                               
                                 hidden_states=((No                                           

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),loss = crf.loss,metrics = crf.accuracy)
num_train_epochs = 50


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor = 'viterbi_accuracy',
    patience = 3,
    restore_best_weights = True
    )

In [ ]:
model.load_weights("/content/drive/MyDrive/Nhóm - Tiến + Quý + Khanh + Văn/IE403 - Khai thác dữ liệu truyền thông xã hội/Đồ án/weights/foor.h5")

In [ ]:
model.fit(
    trainning,
    validation_data = trainning_dev,
    epochs = 4,
    callbacks = early_stop,
    verbose = 1
)

Epoch 1/4


682/682 [==============================] - 1331s 2s/step - loss: 29.0294 - viterbi_accuracy: 0.9701 - val_loss: 783.6747 - val_viterbi_accuracy: 0.9604
Epoch 2/4
682/682 [==============================] - 1212s 2s/step - loss: 24.7705 - viterbi_accuracy: 0.9747 - val_loss: 781.6897 - val_viterbi_accuracy: 0.9609
Epoch 3/4
682/682 [==============================] - 1212s 2s/step - loss: 21.6078 - viterbi_accuracy: 0.9775 - val_loss: 779.9212 - val_viterbi_accuracy: 0.9610
Epoch 4/4
682/682 [==============================] - 1265s 2s/step - loss: 17.9270 - viterbi_accuracy: 0.9810 - val_loss: 778.1384 - val_viterbi_accuracy: 0.9622


# EVALUATION

In [ ]:
test_data = LoadData("/content/drive/MyDrive/dataset/test.jsonl")
X_test,y_test = test_data.load()
X_test,y_test = AlignLabel().transform(X_test,y_test,rdrsegmenter,SEP)
test_data = toPretrainFormat(X_test,y_test,tokenizer,BATCH_SIZE,)

<ipython-input-12-ed114fe53922>:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


In [ ]:
X_test[:5]

array(['sp ổn , mỗi_tội vân tay lúc nhận lúc không , nhân_viên nhiệt_tình , pin trâu , cả đêm tụt 1',
       'mua cho mẹ sài nên củng không đòi_hỏi gì nhiều , máy đẹp camera siêu ảo , thử chiến game củng ok , pin sài dc 2 ngày với luot wep xem fim , nhân_viên tgd an_minh kg phục_vụ qua nhiệt_tình cho 5 *',
       'máy xài tốt , mượt , sạc rất nhanh , pin trâu , mình dùng tác_vụ bình_thường zalo , fb , youtube thì được 1 ngày rưỡi . camera thì đẹp ảo .',
       'mình mới mua . mình thấy mẫu đẹp pin trâu cảm_ứng mượt được em nhân_viên đmx tư_vấn rất nhiệt_tình',
       'máy sài rất êm mượt mà còn thắc_mắc iphone7 plus có chống nước ko bị rơi xuống_nước 1 lần rồi và ko sao cả'],
      dtype='<U770')

In [ ]:
y_temp = []
for i in range(len(y_test)):
  y_t = AlignLabel().tokenize_and_alignlabel(X_test[i],y_test[i],tag2idx,tokenizer)
  y_temp.append(y_t)
y_temp = np.asarray(y_temp)
# y_temp = y_temp.reshape(1,-1)[0]
y_temp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
def pred2tag(y):
  y = y.astype('object')
  for row in range(y.shape[0]):
    for col in range(y.shape[1]):
      y[row][col] = idx2tag[y[row][col]]
  return y

In [ ]:
true = pred2tag(y_temp)
tag = np.array(tag2idx.keys())

In [ ]:
!pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
aspect = np.array(["SCREEN","CAMERA","FEATURES","BATTERY","PERFORMANCE","STORAGE","DESIGN","PRICE","GENERAL","SER&ACC"])
func_add_pola = lambda aspect,pola: [aspect[i] + "#" + pola for i in range(len(aspect))]

aspect_pos = func_add_pola(aspect,"POSITIVE")
aspect_neu = func_add_pola(aspect,"NEUTRAL")
aspect_neg = func_add_pola(aspect,"NEGATIVE")

tags = np.concatenate([aspect_pos,aspect_neu,aspect_neg])

## XLM-RoBERTa

In [ ]:
PRETRAINED_MODEL = "xlm-roberta-base"
model = pretrained_model(PRETRAINED_MODEL)

All PyTorch model weights were used when initializing TFXLMRobertaForTokenClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
y_test_pred = np.argmax(model.predict(test_data,batch_size = BATCH_SIZE),axis=-1)
# y_test_pred = y_test_pred.reshape((1,-1))[0]

279/279 [==============================] - 174s 607ms/step


In [ ]:
pred = pred2tag(y_test_pred)

In [ ]:
aspect_true = true.copy()
aspect_pred = pred.copy()

for i in range(aspect_true.shape[0]):
  aspect_true[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_true[i]))

for i in range(aspect_pred.shape[0]):
  aspect_pred[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_pred[i]))


In [ ]:
# https://github.com/MantisAI/nervaluate
from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=tags, loader="list")

results, results_by_tag = evaluator.evaluate()


In [ ]:
results

{'ent_type': {'correct': 4501,
  'incorrect': 916,
  'partial': 0,
  'missed': 422,
  'spurious': 2119,
  'possible': 5839,
  'actual': 7536,
  'precision': 0.5972664543524416,
  'recall': 0.7708511731460866,
  'f1': 0.6730467289719626},
 'partial': {'correct': 3345,
  'incorrect': 0,
  'partial': 2072,
  'missed': 422,
  'spurious': 2119,
  'possible': 5839,
  'actual': 7536,
  'precision': 0.5813428874734607,
  'recall': 0.7502997088542559,
  'f1': 0.6551028037383179},
 'strict': {'correct': 3077,
  'incorrect': 2340,
  'partial': 0,
  'missed': 422,
  'spurious': 2119,
  'possible': 5839,
  'actual': 7536,
  'precision': 0.4083067940552017,
  'recall': 0.5269737968830279,
  'f1': 0.46011214953271035},
 'exact': {'correct': 3345,
  'incorrect': 2072,
  'partial': 0,
  'missed': 422,
  'spurious': 2119,
  'possible': 5839,
  'actual': 7536,
  'precision': 0.44386942675159236,
  'recall': 0.5728720671347833,
  'f1': 0.5001869158878505}}

## PhoBERTv1

In [ ]:
PRETRAINED_MODEL = "vinai/phobert-base"
inputs = {
    "input_ids" : Input(shape = (MAX_LEN),dtype = 'int32', name = 'input_ids'),
    # "token_type_ids": Input(shape = (MAX_LEN),dtype = 'int32', name = 'token_type_ids'),
    "attention_mask": Input(shape = (MAX_LEN),dtype = 'int32', name = 'attention_mask')
}

model = transformers.TFAutoModelForTokenClassification.from_pretrained(PRETRAINED_MODEL,output_hidden_states=True)
features = model(inputs)[-1]

concat = tf.concat(features[-2:],2)
dropout = Dropout(0.1) (concat)
bi_lstm = Bidirectional(LSTM(768*2,return_sequences=True),merge_mode='concat') (dropout)

output = TimeDistributed(Dense(len(tag2idx),activation = 'relu')) (bi_lstm)

crf = CRF(len(tag2idx))
output = crf(output)
model = Model(inputs = inputs,outputs = output)


All model checkpoint layers were used when initializing TFRobertaForTokenClassification.

Some layers of TFRobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
y_test_pred = np.argmax(model.predict(test_data,batch_size = BATCH_SIZE),axis=-1)
# y_test_pred = y_test_pred.reshape((1,-1))[0]

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


279/279 [==============================] - 161s 555ms/step


In [ ]:
pred = pred2tag(y_test_pred)

In [ ]:
aspect_true = true.copy()
aspect_pred = pred.copy()

for i in range(aspect_true.shape[0]):
  aspect_true[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_true[i]))

for i in range(aspect_pred.shape[0]):
  aspect_pred[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_pred[i]))


In [ ]:
# https://github.com/MantisAI/nervaluate
from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=tags, loader="list")

results, results_by_tag = evaluator.evaluate()


In [ ]:
results

{'ent_type': {'correct': 4443,
  'incorrect': 1002,
  'partial': 0,
  'missed': 394,
  'spurious': 2186,
  'possible': 5839,
  'actual': 7631,
  'precision': 0.5822303760974971,
  'recall': 0.7609179654050351,
  'f1': 0.6596881959910913},
 'partial': {'correct': 3398,
  'incorrect': 0,
  'partial': 2047,
  'missed': 394,
  'spurious': 2186,
  'possible': 5839,
  'actual': 7631,
  'precision': 0.5794129209802122,
  'recall': 0.7572358280527488,
  'f1': 0.6564959168522643},
 'strict': {'correct': 3155,
  'incorrect': 2290,
  'partial': 0,
  'missed': 394,
  'spurious': 2186,
  'possible': 5839,
  'actual': 7631,
  'precision': 0.413445157908531,
  'recall': 0.5403322486727179,
  'f1': 0.4684484038604306},
 'exact': {'correct': 3398,
  'incorrect': 2047,
  'partial': 0,
  'missed': 394,
  'spurious': 2186,
  'possible': 5839,
  'actual': 7631,
  'precision': 0.4452889529550518,
  'recall': 0.5819489638636753,
  'f1': 0.50452858203415}}

## PhoBERTv2

In [ ]:
y_test_pred = np.argmax(model.predict(test_data,batch_size = BATCH_SIZE),axis=-1)
# y_test_pred = y_test_pred.reshape((1,-1))[0]

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


279/279 [==============================] - 145s 501ms/step


In [ ]:
pred = pred2tag(y_test_pred)

In [ ]:
pred

array([['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ...,
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O']], dtype=object)

In [ ]:
for row in range(len(pred)):
  count = 0
  for i in range(MAX_LEN):
    if true[row][i] != pred[row][i]:
      count += 1
  if count >= 20:
    print(row,X_test[row])

5 sản_phẩm qua tệ sai mấy lời liên_tục sắc không võ_cử sắc thì sắc không được . đem sắc mấy khác thì lại vô bin.còn đem về đung mài sắc không . lâu_lâu mấy sắc không . tật mây_khói đông lại thì sắc vô . đem lên thế_giới gỉ đồng thì kêu đem lên hàng bạo_hành . mình đó toàn_diện mấy xanh với thời giới gỉ đồng . mã_lực này bạo_hành mấy của tg đồng kẻm làm . mình không ủng_hộ nữa đâu
9 mới mua 3 ngày máy củ đã qua sử_dụng với giá 8t5 bảo_hành 10 tháng theo đánh_giá giá tiền này thì dòng máy này quá ok , vân tay thì hơi chậm tại vì nếu nhanh thì màn_hình phải sạch ko dính vân tay camera đẹp pin sạc siu nhanh quay video chống rung tốt chụp ảnh thì ok vì mình ích chụp ảnh với giá tiền mình bỏ ra như_vậy thấy là quá ok rồi dc thêm bút spen còn nhận khuôn_mặt mình thấy rất là nhanh và nhạy sao thấy ai củng chê nhỉ mình thấy bao nhanh mà
11 mới mua máy . pin cũng mau hết . xai 4g nóng may . mạng cũng ổn trong tầm giá . man hình kg đẹp bằng super . bấm chưa quen phím điều hướng . tính mua redmi5s

In [ ]:
k = 1193
for word, token_true, token_pred in zip(tokenizer.convert_ids_to_tokens(tokenizer(X_test[k])['input_ids']),true[k],pred[k]):
  # if word != '<pad>':
  print(word, "=>", token_true ,"=>", token_pred)

<s> => O => O
chụp => O => O
hình => O => O
quá => O => O
tệ => O => O
. => O => O
không => B-GENERAL#NEGATIVE => O
đáng => I-GENERAL#NEGATIVE => O
với => I-GENERAL#NEGATIVE => O
giá_trị => I-GENERAL#NEGATIVE => O
bỏ => I-GENERAL#NEGATIVE => O
ra@@ => I-GENERAL#NEGATIVE => O
.@@ => I-GENERAL#NEGATIVE => O
mọi => I-GENERAL#NEGATIVE => O
ng => I-GENERAL#NEGATIVE => O
cân => I-GENERAL#NEGATIVE => O
nhẵ@@ => I-GENERAL#NEGATIVE => O
c => I-GENERAL#NEGATIVE => O
tr@@ => I-GENERAL#NEGATIVE => O
k => I-GENERAL#NEGATIVE => O
khi => I-GENERAL#NEGATIVE => O
mu@@ => I-GENERAL#NEGATIVE => O
a.@@ => I-GENERAL#NEGATIVE => O
quá => I-GENERAL#NEGATIVE => O
tệ => I-GENERAL#NEGATIVE => O
với => I-GENERAL#NEGATIVE => O
số => I-GENERAL#NEGATIVE => I-GENERAL#NEGATIVE
tiền => I-GENERAL#NEGATIVE => I-GENERAL#NEGATIVE
trên => I-GENERAL#NEGATIVE => O
13@@ => I-GENERAL#NEGATIVE => I-GENERAL#NEGATIVE
tr => I-GENERAL#NEGATIVE => I-GENERAL#NEGATIVE
</s> => O => O


In [ ]:
aspect_true = true.copy()
aspect_pred = pred.copy()

for i in range(aspect_true.shape[0]):
  aspect_true[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_true[i]))

for i in range(aspect_pred.shape[0]):
  aspect_pred[i] = list(map(lambda x: x[:x.find("#")] if len(x[:x.find("#")]) >= 2 else "0",aspect_pred[i]))


In [ ]:
# https://github.com/MantisAI/nervaluate
from nervaluate import Evaluator

evaluator = Evaluator(true, pred, tags=tags, loader="list")

results, results_by_tag = evaluator.evaluate()


In [ ]:
results

{'ent_type': {'correct': 4649,
  'incorrect': 809,
  'partial': 0,
  'missed': 381,
  'spurious': 1892,
  'possible': 5839,
  'actual': 7350,
  'precision': 0.632517006802721,
  'recall': 0.7961979791060113,
  'f1': 0.7049814239138675},
 'partial': {'correct': 3618,
  'incorrect': 0,
  'partial': 1840,
  'missed': 381,
  'spurious': 1892,
  'possible': 5839,
  'actual': 7350,
  'precision': 0.6174149659863946,
  'recall': 0.7771878746360679,
  'f1': 0.6881492152551368},
 'strict': {'correct': 3371,
  'incorrect': 2087,
  'partial': 0,
  'missed': 381,
  'spurious': 1892,
  'possible': 5839,
  'actual': 7350,
  'precision': 0.4586394557823129,
  'recall': 0.5773248843980133,
  'f1': 0.5111835620592917},
 'exact': {'correct': 3618,
  'incorrect': 1840,
  'partial': 0,
  'missed': 381,
  'spurious': 1892,
  'possible': 5839,
  'actual': 7350,
  'precision': 0.4922448979591837,
  'recall': 0.6196266483986984,
  'f1': 0.548639017362954}}

In [ ]:
results_by_tag

{'SCREEN#POSITIVE': {'ent_type': {'correct': 132,
   'incorrect': 14,
   'partial': 0,
   'missed': 7,
   'spurious': 22,
   'possible': 153,
   'actual': 168,
   'precision': 0.7857142857142857,
   'recall': 0.8627450980392157,
   'f1': 0.8224299065420562},
  'partial': {'correct': 122,
   'incorrect': 0,
   'partial': 24,
   'missed': 7,
   'spurious': 22,
   'possible': 153,
   'actual': 168,
   'precision': 0.7976190476190477,
   'recall': 0.8758169934640523,
   'f1': 0.8348909657320872},
  'strict': {'correct': 114,
   'incorrect': 32,
   'partial': 0,
   'missed': 7,
   'spurious': 22,
   'possible': 153,
   'actual': 168,
   'precision': 0.6785714285714286,
   'recall': 0.7450980392156863,
   'f1': 0.7102803738317757},
  'exact': {'correct': 122,
   'incorrect': 24,
   'partial': 0,
   'missed': 7,
   'spurious': 22,
   'possible': 153,
   'actual': 168,
   'precision': 0.7261904761904762,
   'recall': 0.7973856209150327,
   'f1': 0.7601246105919004}},
 'CAMERA#POSITIVE': {'ent_

## RANDOM SENTENCE

In [ ]:
sample_sentence = "máy xài tốt , mượt , sạc rất nhanh , pin trâu , mình dùng tác_vụ bình_thường zalo , fb , youtube thì được 1 ngày rưỡi . camera thì đẹp ảo ."
sample_sentence = [" ".join(s) for s in rdrsegmenter.tokenize(sample_sentence)]
sample_sentence = " ".join(sample_sentence)
sample_sentence

'máy xài tốt , mượt , sạc rất nhanh , pin trâu , mình dùng tác _ vụ bình _ thường zalo , fb , youtube thì được 1 ngày rưỡi . camera thì đẹp ảo .'

In [ ]:
tokenized = tokenizer(convert_unicode(TextNormalize().normalize(sample_sentence)),add_special_tokens = True,padding = 'max_length',max_length = MAX_LEN,truncation=True,)
tokenized

{'input_ids': [0, 558, 8480, 167, 4, 11537, 4, 3941, 59, 485, 4, 2179, 2959, 4, 68, 175, 18116, 2926, 178, 2404, 2926, 311, 1791, 22818, 4, 1866, 2774, 4, 31167, 54, 11, 99, 43, 7164, 5, 1901, 54, 258, 2156, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenizer.decode(tokenized['input_ids'])

'<s> máy hình chưa chưa tới 1gb, chỉ tải vài ứng_dụng cơ_bản thôi máy lag rồi, chỉ có 2 điểm cảm_ứng trên máy, facebok lướt không được một_vài phút out ra. game thì khỏi nói chả có nào chiến được. android 8.1 cho vui chứ tải game hay ứng_dụng về để chưng thôi, dùng không nổi. thề, thà để 1tr mua cái qua tay cũ xíu vẫn ngon hơn cái máy này, mua được 1 tuần thay liền ấm_ức chịu không nổi</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
features = {i : [[tokenized[i]]] for i in tokenizer.model_input_names}

In [ ]:
te = Dataset.from_tensor_slices(features)
te

<_TensorSliceDataset element_spec={'input_ids': TensorSpec(shape=(1, 256), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(1, 256), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(1, 256), dtype=tf.int32, name=None)}>

In [ ]:
y_pred = model.predict(te,batch_size=1)
y_pred = np.argmax(y_pred,axis=-1)
y_pred

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1/1 [==============================] - 3s 3s/step


array([[ 0,  0,  0,  0,  0,  5,  0,  4, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34,  0,  2, 32, 32, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
for word, token_pred in zip(tokenized['input_ids'],y_pred[0]):
  word = tokenizer.convert_ids_to_tokens(word)
  if word != '<pad>':
    print(word, "=>", idx2tag[token_pred])

<s> => O
máy => O
xài => O
tốt => O
, => O
mượt => B-PERFORMANCE#POSITIVE
, => O
sạc => B-BATTERY#POSITIVE
rất => I-BATTERY#POSITIVE
nhanh => I-BATTERY#POSITIVE
, => I-BATTERY#POSITIVE
pin => I-BATTERY#POSITIVE
trâu => I-BATTERY#POSITIVE
, => I-BATTERY#POSITIVE
mình => I-BATTERY#POSITIVE
dùng => I-BATTERY#POSITIVE
tác => I-BATTERY#POSITIVE
_ => I-BATTERY#POSITIVE
vụ => I-BATTERY#POSITIVE
bình => I-BATTERY#POSITIVE
_ => I-BATTERY#POSITIVE
thường => I-BATTERY#POSITIVE
z@@ => I-BATTERY#POSITIVE
alo => I-BATTERY#POSITIVE
, => I-BATTERY#POSITIVE
f@@ => I-BATTERY#POSITIVE
b => I-BATTERY#POSITIVE
, => I-BATTERY#POSITIVE
youtube => I-BATTERY#POSITIVE
thì => I-BATTERY#POSITIVE
được => I-BATTERY#POSITIVE
1 => I-BATTERY#POSITIVE
ngày => I-BATTERY#POSITIVE
rưỡi => I-BATTERY#POSITIVE
. => O
camera => B-CAMERA#POSITIVE
thì => I-CAMERA#POSITIVE
đẹp => I-CAMERA#POSITIVE
ảo => I-CAMERA#POSITIVE
. => O
</s> => O
